# Paquetes

In [ ]:
!pip3 install pip chromadb gradio langchain sentence_transformers pypdf --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 594.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

# Configuracion Embeddings

In [ ]:
EMBEDDING_MODEL = "intfloat/multilingual-e5-large" # https://huggingface.co/models?library=sentence-transformers
EMBEDDINGS_NORMALIZE = True
EMBEDDINGS_QUERY_PROMPT = "query: "

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

encode_kwargs = {'normalize_embeddings': EMBEDDINGS_NORMALIZE}

embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL,encode_kwargs=encode_kwargs)

ModuleNotFoundError: No module named 'langchain_community'

# Configuracion DB

In [ ]:
DB = "chromadb" # chromadb | faiss

In [ ]:
import pip

if DB == "chromadb":
    pip.main(["install","chromadb","--upgrade","--quiet"])
    from langchain_community.vectorstores import Chroma as VectorStore
elif DB == "faiss":
    pip.main(["install","faiss-cpu","--upgrade","--quiet"])
    from langchain_community.vectorstores import FAISS as VectorStore
else:
    raise ("Invalid vector store")

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv


# Configuracion Splitter

In [ ]:
SPLITTER = "recursive" # recursive |

TEXT_SPLITTER_CHUNKSIZE = 1000 # Tamano en caracteres del bloque de texto
TEXT_SPLITTER_OVERLAP = 200    # Tamano en caracteres del la superposicion entre bloques
TEXT_SPLITTER_SEPARATORS = ["\n\n","\n","."," "] # Separadores

In [ ]:
if SPLITTER == "recursive":
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=TEXT_SPLITTER_CHUNKSIZE,
                                                   chunk_overlap=TEXT_SPLITTER_OVERLAP,
                                                   separators=TEXT_SPLITTER_SEPARATORS)
else:
    raise ("Invalid splitter")

# Datos

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import YoutubeLoader

### Youtube libs

In [ ]:
!pip install youtube-transcript-api pytube --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.2 MB/s eta 0:00:00


In [ ]:
PDF_FILES = []
YOUTUBE_URLS = ["https://www.youtube.com/watch?v=Z3aWHMg92_U"]

In [ ]:
raw_docs = []

In [ ]:
for pdf in PDF_FILES:
    raw_docs += PyPDFLoader(pdf).load()

for vid in YOUTUBE_URLS:
    raw_docs += YoutubeLoader.from_youtube_url(
    vid,
    add_video_info=True,
    language=["en", "id"],
    translation="es",
).load()


In [ ]:
documents = text_splitter.split_documents(raw_docs)
db = VectorStore.from_documents(documents, embeddings)

In [ ]:
retriever = db.as_retriever()

# Gradio Server

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(likeable=True,layout="bubble",height="70%")
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    shutdown = gr.Button(value="Shutdown")
    shutdown.click(lambda _:demo.close())

    last_docs = gr.State(None)
    last_docs_index = gr.State(0)

    def respond(message, chat_history, last_docs, last_docs_index):
        if not chat_history:
            chat_history.append((message, "Hola, en que puedo ayudarte?"))
            return "", chat_history, last_docs, last_docs_index

        if last_docs:
            if "si" in message:
                last_docs = None
                chat_history.append((message, "Genial! Si tenes otra consulta decime."))
                return "", chat_history, last_docs, last_docs_index

            last_docs_index += 1

            if len(last_docs) <= last_docs_index:
                chat_history.append((message, "Lamento no haber encontrado una respuesta. Intenta haciendo la pregunta de forma distinta."))
                last_docs = None
                return "", chat_history, last_docs, last_docs_index

            doc = last_docs[last_docs_index]
            bot_message = f"Tambien habia encontrado esto:\n\n``` {doc.page_content} \n```\n\nEsta en {doc.metadata['source']} en la pagina {doc.metadata['page']}. ¿Esto si te sirvio?"
            chat_history.append((message, bot_message))
            return "", chat_history, last_docs, last_docs_index

        docs = retriever.get_relevant_documents(EMBEDDINGS_QUERY_PROMPT+message.lower())
        last_docs = docs

        last_docs_index = 0
        doc = docs[last_docs_index]

        bot_message = f"Encontre esto:\n\n``` {doc.page_content} \n```\n\nEsta en {doc.metadata['source']} en la pagina {doc.metadata['page']}. ¿Te sirvio?"

        chat_history.append((message, bot_message))
        return "", chat_history, last_docs, last_docs_index

    msg.submit(respond, [msg, chatbot, last_docs, last_docs_index], [msg, chatbot,last_docs, last_docs_index])

if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", server_port=7860)

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:985: UserWarning: Expected 1 arguments for function <function <lambda> at 0x7f8c29019cf0>, received 0.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:989: UserWarning: Expected at least 1 arguments for function <function <lambda> at 0x7f8c29019cf0>, received 0.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ce449898d106de2358.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
demo.close()

Closing server running on port: 7860


In [ ]:
retriever.get_relevant_documents(EMBEDDINGS_QUERY_PROMPT+"why did izumo forge the swords?".lower())

WARNING: Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(page_content='¿Aún recuerdas por qué Izumo forjó estas espadas? Porque en tiempos pasados, cuando los Yaoyorozu-no-Kami descendieron aquí, provocaron muerte y desastre en este mundo y su gente. Para salvar el mundo, Izumo partió 70.033 espadas y las forjó en los 12 Centinelas. La primera fue la "Verdad", forjada por el Soberano de la Revelación. Podría enseñar a los mortales el camino de la ley y crear milagros a partir de la deconstrucción de todas las cosas. El segundo fue "Sky", forjado por el Eternal Zenith. Podría formar una barrera desde el firmamento e impedir el paso de los innumerables dioses de la miseria. El tercero fue "Aullido", capaz de convocar relámpagos para desgarrar los cielos y meteoritos para hacer llover el castigo divino. El cuarto fue "Mist", capaz de romper la tierra y rasgar las nubes con viento y lluvia cortantes. El quinto fue "Frost", congelando la tierra en un instante. El sexto fue "Destino", el inquieto ciclo de la vida y la muerte', metadata={

In [ ]:
retriever.get_relevant_documents(EMBEDDINGS_QUERY_PROMPT+"why did izumo forge the swords?".lower())[0].page_content

WARNING: Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


'¿Aún recuerdas por qué Izumo forjó estas espadas? Porque en tiempos pasados, cuando los Yaoyorozu-no-Kami descendieron aquí, provocaron muerte y desastre en este mundo y su gente. Para salvar el mundo, Izumo partió 70.033 espadas y las forjó en los 12 Centinelas. La primera fue la "Verdad", forjada por el Soberano de la Revelación. Podría enseñar a los mortales el camino de la ley y crear milagros a partir de la deconstrucción de todas las cosas. El segundo fue "Sky", forjado por el Eternal Zenith. Podría formar una barrera desde el firmamento e impedir el paso de los innumerables dioses de la miseria. El tercero fue "Aullido", capaz de convocar relámpagos para desgarrar los cielos y meteoritos para hacer llover el castigo divino. El cuarto fue "Mist", capaz de romper la tierra y rasgar las nubes con viento y lluvia cortantes. El quinto fue "Frost", congelando la tierra en un instante. El sexto fue "Destino", el inquieto ciclo de la vida y la muerte'